In [ ]:
!pip3 install pydicom nibabel matplotlib numpy pandas

# Lectura de archivos DICOM, extracción de metadatos y visualización de imágenes médicas.

In [ ]:
# Lectura archivo DICOM

import pydicom

# Cargar un archivo DICOM
dicom_path = 'inputs/chest_serie/1-01.dcm'
ds = pydicom.dcmread(dicom_path)

# Mostrar información básica
print(ds.__str__())

In [ ]:
# Ejemplo: leer algunos metadatos

print('Patient Name:', ds.PatientName)
print('Patient ID:', ds.PatientID)
print('Study Date:', ds.StudyDate)
print('Modality:', ds.Modality)
print('Manufacturer:', ds.Manufacturer)
print('ImagePositionPatient:', ds.ImagePositionPatient)
print('ImageOrientationPatient:', ds.ImageOrientationPatient)
print('PatientPosition:', ds.PatientPosition)

In [ ]:
# Guardar metadatos en un csv

# Extraer metadatos
metadata = {
    "PatientName": ds.PatientName,
    "PatientID": ds.PatientID,
    "StudyDate": ds.StudyDate,
    "Modality": ds.Modality,
    "Manufacturer": ds.Manufacturer,
    "ImagePositionPatient": ds.ImagePositionPatient,
    "ImageOrientationPatient": ds.ImageOrientationPatient,
    "PatientPosition":  ds.PatientPosition
}

import os
import pandas as pd

# Crear un DataFrame de una fila
df = pd.DataFrame([metadata])

output_path = '/results'
# Guardar en CSV
df.to_csv(os.path.join(output_path, "metadatos_dicom.csv"), index=False)

In [ ]:
# Visualizar imagen DICOM 2D

import matplotlib.pyplot as plt

# Acceder a los datos de imagen
image = ds.pixel_array

# Mostrar la imagen
plt.imshow(image, cmap='gray')
plt.title('Imagen DICOM')
plt.axis('off')
plt.show()


In [ ]:
# Cargar carpeta completa de DICOMs y crear volumen 3D

import numpy as np

import os
import pydicom
import numpy as np
from collections import Counter

# Ruta a la carpeta que contiene los archivos DICOM de un mismo estudio
dicom_folder = 'inputs/chest_serie'

# --------------------------------------------
# Paso 1: Buscar todos los archivos que terminan en .dcm
# --------------------------------------------
dicom_files = [os.path.join(dicom_folder, f) for f in os.listdir(dicom_folder) if f.endswith('.dcm')]

# --------------------------------------------
# Paso 2: Leer todos los archivos DICOM encontrados
# Usamos try-except para evitar que un archivo corrupto interrumpa el proceso
# --------------------------------------------
slices = []
for filepath in dicom_files:
    try:
        dcm = pydicom.dcmread(filepath)  # Leer el archivo DICOM
        slices.append(dcm)
    except Exception as e:
        print(f"Error leyendo {filepath}: {e}")  # Reportar errores sin detener el proceso

# --------------------------------------------
# Paso 3: Filtrar solo las imágenes que contienen la posición en el cuerpo
# 'ImagePositionPatient' indica la coordenada (x, y, z) del primer píxel
# También verificamos que contengan los datos de imagen (pixel_array)
# --------------------------------------------
slices_validas = [s for s in slices if hasattr(s, 'ImagePositionPatient') and hasattr(s, 'pixel_array')]

# --------------------------------------------
# Paso 4: Ordenar las imágenes en función de la coordenada Z
# Esto es importante para reconstruir el volumen 3D en el orden correcto
# --------------------------------------------
def obtener_posicion_z(slice):
    return float(slice.ImagePositionPatient[2])  # Extraemos la coordenada Z

slices_validas.sort(key=obtener_posicion_z)

# --------------------------------------------
# Paso 5: Verificar que todas las imágenes tengan la misma forma (alto x ancho)
# Esto es obligatorio para poder apilar las imágenes en una matriz 3D
# --------------------------------------------
formas = [s.pixel_array.shape for s in slices_validas]
forma_mas_comun = Counter(formas).most_common(1)[0][0]  # Forma más frecuente

# --------------------------------------------
# Paso 6: Filtrar solo aquellas imágenes que tengan la forma más común
# Esto evita errores al apilar imágenes con tamaños distintos
# --------------------------------------------
slices_filtradas = [s for s in slices_validas if s.pixel_array.shape == forma_mas_comun]

# --------------------------------------------
# Paso 7: Crear un volumen 3D apilando las imágenes en el eje Z
# La forma del volumen será (alto, ancho, número de cortes)
# --------------------------------------------
volume_dicom = np.stack([s.pixel_array for s in slices_filtradas], axis=-1)

# --------------------------------------------
# Paso 8: Mostrar el tamaño final del volumen
# --------------------------------------------
print("Tamaño del volumen (alto, ancho, número de cortes):", volume_dicom.shape)


In [ ]:
## Visualizar cortes en diferentes planos

# Corte axial
plt.imshow(volume_dicom[:, :, volume_dicom.shape[2] // 2], cmap='gray')
plt.title('Corte Axial')
plt.axis('off')
plt.show()

# Corte coronal
plt.imshow(volume_dicom[:, volume_dicom.shape[1] // 2, :], cmap='gray')
plt.title('Corte Sagital')
plt.axis('off')
plt.show()

# Corte sagital
plt.imshow(volume_dicom[volume_dicom.shape[0] // 2, :, :], cmap='gray')
plt.title('Corte Coronal')
plt.axis('off')
plt.show()

In [ ]:
## Visualizar cortes en diferentes planos

# Corte axial
plt.imshow(volume_dicom[:, :, volume_dicom.shape[2] // 2], cmap='gray')
plt.title('Corte Axial')
plt.axis('off')
plt.show()

# Corte coronal
plt.imshow(np.rot90(volume_dicom[:, volume_dicom.shape[1] // 2, :]), cmap='gray')
plt.title('Corte Sagital')
plt.axis('off')
plt.show()

# Corte sagital
plt.imshow(np.rot90(volume_dicom[volume_dicom.shape[0] // 2, :, :]), cmap='gray')
plt.title('Corte Coronal')
plt.axis('off')
plt.show()

# Lectura de archivos NIfTI, visualización en distintos planos anatómicos de un volumen, y superposición de la imagen con su segmentación.

In [ ]:
# Código para convertir un volumen DICOM a NIfTI

import numpy as np
import nibabel as nib

# --------------------------------------------
# Paso 1: Construir la matriz affine realista
# Esto define cómo mapear los índices de voxel (i, j, k) al espacio físico real del paciente (en mm)
# --------------------------------------------

# Usamos el primer corte como referencia para extraer la orientación y posición
ref_slice = slices_filtradas[0]

# Origen del volumen: coordenadas físicas del primer píxel del primer corte
origin = np.array(ref_slice.ImagePositionPatient, dtype=float)  # [x0, y0, z0] en mm

# Direcciones de fila y columna (definidas en el espacio del paciente)
orientation = np.array(ref_slice.ImageOrientationPatient, dtype=float)
row_direction = orientation[0:3]  # dirección horizontal en la imagen
col_direction = orientation[3:6]  # dirección vertical en la imagen

# Espaciado del píxel en mm
pixel_spacing = np.array(ref_slice.PixelSpacing, dtype=float)  # [row_spacing, col_spacing]

# Calcular el vector normal al plano (dirección entre cortes, eje Z)
normal_direction = np.cross(row_direction, col_direction)

# Calcular el espaciado entre cortes: mejor usar diferencia real si hay al menos 2 cortes
if len(slices_filtradas) >= 2:
    z0 = float(slices_filtradas[0].ImagePositionPatient[2])
    z1 = float(slices_filtradas[1].ImagePositionPatient[2])
    slice_spacing = abs(z1 - z0)
else:
    slice_spacing = float(ref_slice.SliceThickness)

# Construimos la parte de rotación/escala de la affine (3x3)
# Cada columna representa una dirección del espacio (X, Y, Z)
affine_orientation = np.column_stack([
    col_direction * pixel_spacing[1],   # Dirección X (columnas → izquierda/derecha)
    row_direction * pixel_spacing[0],   # Dirección Y (filas → anterior/posterior)
    normal_direction * slice_spacing    # Dirección Z (entre cortes → superior/inferior)
])

# Creamos la matriz affine completa (4x4), que incluye traslación (origen)
affine = np.eye(4)
affine[:3, :3] = affine_orientation
affine[:3, 3] = origin

# --------------------------------------------
# Paso 5: Crear y guardar el archivo NIfTI
# El volumen se guarda junto con su geometría espacial en formato .nii.gz
# --------------------------------------------

nifti_img = nib.Nifti1Image(volume_dicom.astype(np.int16), affine)
nib.save(nifti_img, 'image.nii.gz')

print("✅ Volumen guardado como NIfTI con orientación espacial real en 'image.nii.gz'")

In [ ]:
# Leer archivo NIfTI

import nibabel as nib

# Cargar un archivo NIfTI
nifti_path = './image.nii.gz'
nifti_img = nib.load(nifti_path)

# Mostrar la forma de los datos
print('Shape:', nifti_img.shape)

# Obtener la imagen en array
nifti_data = nifti_img.get_fdata()

In [ ]:
## Visualizar cortes en diferentes planos

# Corte axial
plt.imshow(nifti_data[:, :, nifti_data.shape[2] // 2], cmap='gray')
plt.title('Corte Axial')
plt.axis('off')
plt.show()

# Corte coronal
plt.imshow(nifti_data[:, nifti_data.shape[1] // 2, :], cmap='gray')
plt.title('Corte Sagital')
plt.axis('off')
plt.show()

# Corte sagital
plt.imshow(nifti_data[nifti_data.shape[0] // 2, :, :], cmap='gray')
plt.title('Corte Coronal')
plt.axis('off')
plt.show()
